# Reddit Scraper and Feature Extraction

## Table of Contents:
* [Libraries](#libraries)
* [Reddit API Extraction Object](#reddit_extract)
* [Reddit Feature Extraction: Sentiment Analysis and Phrase Matching](#reddit_sentiment_and_matching)

### Libraries <a class="anchor" id="libraries"></a>

In [1]:
import pandas as pd
import datetime as dt
import requests
import json
import numpy as np
import nltk
nltk.download('stopwords')
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import TextBlob
import spacy
from spacy.matcher import PhraseMatcher

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Reddit API Extraction Object <a class="anchor" id="reddit_extract"></a>

In [2]:
class Reddit_API_Extraction:
    def __init__(self, subreddit, start_date, end_date):
        self.subreddit=subreddit
        self.start_date=start_date
        self.end_date=end_date
        
    @staticmethod
    def create_date_range(start_date_range, end_date_range):
        date_range_list=pd.date_range(start=f"{start_date_range}",
                                      end=f"{end_date_range}", 
                                      freq='D', 
                                      tz='UTC')
        date_range_timestamp=[int(i.timestamp()) for i in date_range_list]
        
        return date_range_timestamp
    
    @classmethod
    def data_extraction(cls, start_date_range, end_date_range, subreddit):
        date_range_utc=cls.create_date_range(start_date_range=start_date_range, end_date_range=end_date_range)
        
        begin_url='''https://api.pushshift.io/reddit/submission/search/?'''
        sorting_url='''&sort_type=score&sort=desc'''
        limit_url='''&limit=100'''
        subreddit_url=f'''&subreddit={subreddit}'''

        post_score=[]
        post_author=[]
        post_full_link=[]
        post_date_utc=[]
        post_num_comments=[]
        post_permalink=[]
        post_subreddit_subscribers=[]
        post_title=[]
        post_image_url=[]
        post_upvote_ratio=[]
        
        date_index_after=0
        date_index_before=1
        
        progress_counter=0
        
        while date_index_before<len(date_range_utc):
            after_url=f'''after={date_range_utc[date_index_after]}'''
            before_url=f'''&before={date_range_utc[date_index_before]}'''

            reddit_url=begin_url+after_url+before_url+sorting_url+limit_url+subreddit_url

            try:
                data_resp=requests.get(url=reddit_url).json()

                for post in data_resp['data']:
                    try:
                        post_author.append(post['author'])
                    except:
                        post_author.append(np.nan)

                    try:
                        post_score.append(post['score'])
                    except:
                        post_score.append(np.nan)


                    try:
                        post_full_link.append(post['full_link'])
                    except:
                        post_full_link.append(np.nan)


                    try:
                        post_date_utc.append(post['created_utc'])
                    except:
                        post_date_utc.append(np.nan)


                    try:
                        post_num_comments.append(post['num_comments'])
                    except:
                        post_num_comments.append(np.nan)


                    try:
                        post_permalink.append(post['permalink'])
                    except:
                        post_permalink.append(np.nan)

                    try:
                        post_subreddit_subscribers.append(post['subreddit_subscribers'])
                    except:
                        post_subreddit_subscribers.append(np.nan)


                    try:
                        post_title.append(post['title'])
                    except:
                        post_title.append(np.nan)


                    try:
                        post_image_url.append(post['url'])
                    except:
                        post_image_url.append(np.nan)


                    try:
                        post_upvote_ratio.append(post['upvote_ratio'])
                    except:  
                        post_upvote_ratio.append(np.nan)

                date_index_after=date_index_after+1
                date_index_before=date_index_before+1
                
                progress_counter=progress_counter+1
                
                print(f'{progress_counter}/{len(date_range_utc)}')

            except:
                date_index_after=date_index_after
                date_index_before=date_index_before
                progress_counter=progress_counter
                
        reddit_df=pd.DataFrame()
        reddit_df['score']=post_score
        reddit_df['author']=post_author
        reddit_df['full_link']=post_full_link
        reddit_df['utc_date']=post_date_utc
        reddit_df['num_comments']=post_num_comments
        reddit_df['subreddit_loc']=post_permalink
        reddit_df['num_subscribers']=post_subreddit_subscribers
        reddit_df['title']=post_title
        reddit_df['img_url']=post_image_url
        reddit_df['upvote_ratio']=post_upvote_ratio
                
        return reddit_df
    
    def clean_df(self):
        clean_reddit_df=self.data_extraction(start_date_range=self.start_date, 
                                             end_date_range=self.end_date, 
                                             subreddit=self.subreddit)
        
        clean_reddit_df['utc_date']=pd.to_datetime(clean_reddit_df['utc_date'],unit='s')
        
        return clean_reddit_df

In [3]:
reddit_api_extraction=Reddit_API_Extraction(subreddit='sandiego', 
                                            start_date='2020-01-01', 
                                            end_date='2021-01-01')

reddit_data_df=reddit_api_extraction.clean_df()
reddit_data_df.to_csv('gs://dse203_group_project/Data/unstructured_posts/reddit_data.csv', index=False)

1/367
2/367
3/367
4/367
5/367
6/367
7/367
8/367
9/367
10/367
11/367
12/367
13/367
14/367
15/367
16/367
17/367
18/367
19/367
20/367
21/367
22/367
23/367
24/367
25/367
26/367
27/367
28/367
29/367
30/367
31/367
32/367
33/367
34/367
35/367
36/367
37/367
38/367
39/367
40/367
41/367
42/367
43/367
44/367
45/367
46/367
47/367
48/367
49/367
50/367
51/367
52/367
53/367
54/367
55/367
56/367
57/367
58/367
59/367
60/367
61/367
62/367
63/367
64/367
65/367
66/367
67/367
68/367
69/367
70/367
71/367
72/367
73/367
74/367
75/367
76/367
77/367
78/367
79/367
80/367
81/367
82/367
83/367
84/367
85/367
86/367
87/367
88/367
89/367
90/367
91/367
92/367
93/367
94/367
95/367
96/367
97/367
98/367
99/367
100/367
101/367
102/367
103/367
104/367
105/367
106/367
107/367
108/367
109/367
110/367
111/367
112/367
113/367
114/367
115/367
116/367
117/367
118/367
119/367
120/367
121/367
122/367
123/367
124/367
125/367
126/367
127/367
128/367
129/367
130/367
131/367
132/367
133/367
134/367
135/367
136/367
137/367
138/367
139/

### Reddit Feature Extraction: Sentiment Analysis and Phrase Matching <a class="anchor" id="reddit_sentiment_and_matching"></a>

In [2]:
class Reddit_Data_Features:
    def __init__(self):
        self.neighborhoods=["MIRAMAR RANCH NORTH", "TORREY HIGHLANDS", "MISSION BAY", "NORTH CITY", 
                            "LOMA PORTAL", "GASLAMP", "HILLCREST", "SOUTH PARK", "NORTH PARK", "SOUTHCREST", 
                            "TERALTA WEST", "AZALEA/HOLLYWOOD PARK", "MT HOPE","TERALTA EAST", "CHOLLAS VIEW", 
                            "RIDGEVIEW/WEBSTER", "COLINA DEL SOL", "PARADISE HILLS", "BAY TERRACES", 
                            "SORRENTO VALLEY", "SERRA MESA", "RANCHO BERNARDO", "OCEAN CREST", "LA PLAYA",
                            "ROSEVILLE / FLEET RIDGE", "HORTON PLAZA", "CORE-COLUMBIA", "SHERMAN HEIGHTS", 
                            "BURLINGAME", "STOCKTON", "NORMAL HEIGHTS", "EMERALD HILLS", 
                            "REDWOOD VILLAGE/ROLANDO PARK", "ENCANTO", "QUALCOMM", "MIRA MESA", "SAN CARLOS", 
                            "RANCHO PENASQUITOS", "SUNSET CLIFFS", "WOODED AREA", "PACIFIC BEACH", "CARMEL VALLEY",
                            "MISSION HILLS", "CHEROKEE POINT", "CORRIDOR", "SWAN CANYON", "ISLENAIR", 
                            "COLLEGE WEST", "BROADWAY HEIGHTS", "ALLIED GARDENS", "LAKE MURRAY", "UNIVERSITY CITY",
                            "EGGER HIGHLANDS", "TIJUANA RIVER VALLEY", "SAN YSIDRO", "OLD TOWN", "MORENA", 
                            "MIDTOWN", "FAIRMONT VILLAGE", "LINCOLN PARK", "CHOLLAS CREEK", "GRANTVILLE",
                            "DEL CERRO", "CARMEL MOUNTAIN", "SAN PASQUAL", "OCEAN BEACH", "LA JOLLA", 
                            "TORREY PINES", "BAY HO", "HARBORVIEW", "PARK WEST", "LINDA VISTA", "GOLDEN HILL", 
                            "SHELLTOWN", "CASTLE", "VALENCIA PARK", "COLLEGE EAST", "CLAIREMONT MESA EAST", 
                            "TIERRASANTA", "BLACK MOUNTAIN RANCH", "RANCHO ENCANTADA", "MISSION BEACH", 
                            "MIDWAY DISTRICT", "PETCO PARK", "BALBOA PARK", "BARRIO LOGAN", "FOX CANYON", "TALMADGE",
                            "ROLANDO", "OTAY MESA", "POINT LOMA HEIGHTS", "NESTOR", "BAY PARK", "MARINA", 
                            "CORTEZ", "EAST VILLAGE", "UNIVERSITY HEIGHTS", "GRANT HILL", "ADAMS NORTH", 
                            "KENSINGTON", "OAK PARK", "NORTH CLAIREMONT", "BORDER", "JAMACHA LOMITA", 
                            "TORREY PRESERVE", "DEL MAR HEIGHTS", "PALM CITY", "OTAY MESA WEST",
                            "LITTLE ITALY", "MISSION VALLEY WEST", "LOGAN HEIGHTS", "MISSION VALLEY EAST", 
                            "FAIRMONT PARK", "EL CERRITO", "ALTA VISTA", "CLAIREMONT MESA WEST", "BIRDLAND", 
                            "MIRAMAR", "KEARNY MESA", "SABRE SPRINGS"]
    
    @staticmethod
    def reddit_sentiment_analysis(data):
        data['title_clean']=data['title'].str.strip()
        data['title_clean']=data['title_clean'].apply(lambda x: " ".join(x.lower() for x in x.split()))
        data['title_clean']=data['title_clean'].str.replace('[!,*)@#%(&$_?.^]','')

        stop=stopwords.words('english')
        data['title_clean']=data['title_clean'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

        st = PorterStemmer()
        data['title_clean']=data['title_clean'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

        def senti(x):
            sentiment_score=TextBlob(x).sentiment[0]  
            return sentiment_score

        data['sentiment']=data['title_clean'].apply(senti)
        
        return data
    
    def phrase_matching(self, data):
        data=self.reddit_sentiment_analysis(data=data)
        
        neighborhoods=[x.lower() for x in self.neighborhoods]
        
        nlp = spacy.load("en_core_web_sm")
        matcher = PhraseMatcher(nlp.vocab)

        patterns = [nlp.make_doc(text) for text in neighborhoods]
        matcher.add("TerminologyList", patterns)

        reddit_titles=data['title'].values.tolist()
        length_titles=len(data['title'].values.tolist())
        i=1

        store_matches=[]
        for title in reddit_titles:
            store_span=[]
            doc = nlp(title)
            matches = matcher(doc)
            for match_id, start, end in matches:
                span = doc[start:end]
                store_span.append(span.text)

            store_matches.append(store_span)
            print(f'{i}/{length_titles}')
            i=i+1
            
        
        neighborhood_matches=pd.DataFrame(store_matches)
        neighborhood_matches=neighborhood_matches.fillna(value=np.nan)
        neighborhood_matches.columns=['neighbourhood_match1', 'neighbourhood_match2']

        reddit_neighborhood=data.merge(neighborhood_matches, how='left', left_index=True, right_index=True)
        reddit_neighborhood.dropna(subset=['neighbourhood_match1', 'neighbourhood_match2'], how='all', inplace=True)
        
        return reddit_neighborhood

In [3]:
data=pd.read_csv('gs://dse203_group_project/Data/unstructured_posts/reddit_data.csv')

reddit_data_features=Reddit_Data_Features()
final_reddit_data=reddit_data_features.phrase_matching(data=data)
final_reddit_data.to_csv('gs://dse203_group_project/Data/unstructured_posts/final_reddit_data.csv', index=False)

/tmp/ipykernel_15570/258380694.py:33: FutureWarning: The default value of regex will change from True to False in a future version.
  data['title_clean']=data['title_clean'].str.replace('[!,*)@#%(&$_?.^]','')


1/18709
2/18709
3/18709
4/18709
5/18709
6/18709
7/18709
8/18709
9/18709
10/18709
11/18709
12/18709
13/18709
14/18709
15/18709
16/18709
17/18709
18/18709
19/18709
20/18709
21/18709
22/18709
23/18709
24/18709
25/18709
26/18709
27/18709
28/18709
29/18709
30/18709
31/18709
32/18709
33/18709
34/18709
35/18709
36/18709
37/18709
38/18709
39/18709
40/18709
41/18709
42/18709
43/18709
44/18709
45/18709
46/18709
47/18709
48/18709
49/18709
50/18709
51/18709
52/18709
53/18709
54/18709
55/18709
56/18709
57/18709
58/18709
59/18709
60/18709
61/18709
62/18709
63/18709
64/18709
65/18709
66/18709
67/18709
68/18709
69/18709
70/18709
71/18709
72/18709
73/18709
74/18709
75/18709
76/18709
77/18709
78/18709
79/18709
80/18709
81/18709
82/18709
83/18709
84/18709
85/18709
86/18709
87/18709
88/18709
89/18709
90/18709
91/18709
92/18709
93/18709
94/18709
95/18709
96/18709
97/18709
98/18709
99/18709
100/18709
101/18709
102/18709
103/18709
104/18709
105/18709
106/18709
107/18709
108/18709
109/18709
110/18709
111/1870

851/18709
852/18709
853/18709
854/18709
855/18709
856/18709
857/18709
858/18709
859/18709
860/18709
861/18709
862/18709
863/18709
864/18709
865/18709
866/18709
867/18709
868/18709
869/18709
870/18709
871/18709
872/18709
873/18709
874/18709
875/18709
876/18709
877/18709
878/18709
879/18709
880/18709
881/18709
882/18709
883/18709
884/18709
885/18709
886/18709
887/18709
888/18709
889/18709
890/18709
891/18709
892/18709
893/18709
894/18709
895/18709
896/18709
897/18709
898/18709
899/18709
900/18709
901/18709
902/18709
903/18709
904/18709
905/18709
906/18709
907/18709
908/18709
909/18709
910/18709
911/18709
912/18709
913/18709
914/18709
915/18709
916/18709
917/18709
918/18709
919/18709
920/18709
921/18709
922/18709
923/18709
924/18709
925/18709
926/18709
927/18709
928/18709
929/18709
930/18709
931/18709
932/18709
933/18709
934/18709
935/18709
936/18709
937/18709
938/18709
939/18709
940/18709
941/18709
942/18709
943/18709
944/18709
945/18709
946/18709
947/18709
948/18709
949/18709
950/18709


1623/18709
1624/18709
1625/18709
1626/18709
1627/18709
1628/18709
1629/18709
1630/18709
1631/18709
1632/18709
1633/18709
1634/18709
1635/18709
1636/18709
1637/18709
1638/18709
1639/18709
1640/18709
1641/18709
1642/18709
1643/18709
1644/18709
1645/18709
1646/18709
1647/18709
1648/18709
1649/18709
1650/18709
1651/18709
1652/18709
1653/18709
1654/18709
1655/18709
1656/18709
1657/18709
1658/18709
1659/18709
1660/18709
1661/18709
1662/18709
1663/18709
1664/18709
1665/18709
1666/18709
1667/18709
1668/18709
1669/18709
1670/18709
1671/18709
1672/18709
1673/18709
1674/18709
1675/18709
1676/18709
1677/18709
1678/18709
1679/18709
1680/18709
1681/18709
1682/18709
1683/18709
1684/18709
1685/18709
1686/18709
1687/18709
1688/18709
1689/18709
1690/18709
1691/18709
1692/18709
1693/18709
1694/18709
1695/18709
1696/18709
1697/18709
1698/18709
1699/18709
1700/18709
1701/18709
1702/18709
1703/18709
1704/18709
1705/18709
1706/18709
1707/18709
1708/18709
1709/18709
1710/18709
1711/18709
1712/18709
1713/18709

2382/18709
2383/18709
2384/18709
2385/18709
2386/18709
2387/18709
2388/18709
2389/18709
2390/18709
2391/18709
2392/18709
2393/18709
2394/18709
2395/18709
2396/18709
2397/18709
2398/18709
2399/18709
2400/18709
2401/18709
2402/18709
2403/18709
2404/18709
2405/18709
2406/18709
2407/18709
2408/18709
2409/18709
2410/18709
2411/18709
2412/18709
2413/18709
2414/18709
2415/18709
2416/18709
2417/18709
2418/18709
2419/18709
2420/18709
2421/18709
2422/18709
2423/18709
2424/18709
2425/18709
2426/18709
2427/18709
2428/18709
2429/18709
2430/18709
2431/18709
2432/18709
2433/18709
2434/18709
2435/18709
2436/18709
2437/18709
2438/18709
2439/18709
2440/18709
2441/18709
2442/18709
2443/18709
2444/18709
2445/18709
2446/18709
2447/18709
2448/18709
2449/18709
2450/18709
2451/18709
2452/18709
2453/18709
2454/18709
2455/18709
2456/18709
2457/18709
2458/18709
2459/18709
2460/18709
2461/18709
2462/18709
2463/18709
2464/18709
2465/18709
2466/18709
2467/18709
2468/18709
2469/18709
2470/18709
2471/18709
2472/18709

...

15569/18709
15570/18709
15571/18709
15572/18709
15573/18709
15574/18709
15575/18709
15576/18709
15577/18709
15578/18709
15579/18709
15580/18709
15581/18709
15582/18709
15583/18709
15584/18709
15585/18709
15586/18709
15587/18709
15588/18709
15589/18709
15590/18709
15591/18709
15592/18709
15593/18709
15594/18709
15595/18709
15596/18709
15597/18709
15598/18709
15599/18709
15600/18709
15601/18709
15602/18709
15603/18709
15604/18709
15605/18709
15606/18709
15607/18709
15608/18709
15609/18709
15610/18709
15611/18709
15612/18709
15613/18709
15614/18709
15615/18709
15616/18709
15617/18709
15618/18709
15619/18709
15620/18709
15621/18709
15622/18709
15623/18709
15624/18709
15625/18709
15626/18709
15627/18709
15628/18709
15629/18709
15630/18709
15631/18709
15632/18709
15633/18709
15634/18709
15635/18709
15636/18709
15637/18709
15638/18709
15639/18709
15640/18709
15641/18709
15642/18709
15643/18709
15644/18709
15645/18709
15646/18709
15647/18709
15648/18709
15649/18709
15650/18709
15651/18709
1565

16273/18709
16274/18709
16275/18709
16276/18709
16277/18709
16278/18709
16279/18709
16280/18709
16281/18709
16282/18709
16283/18709
16284/18709
16285/18709
16286/18709
16287/18709
16288/18709
16289/18709
16290/18709
16291/18709
16292/18709
16293/18709
16294/18709
16295/18709
16296/18709
16297/18709
16298/18709
16299/18709
16300/18709
16301/18709
16302/18709
16303/18709
16304/18709
16305/18709
16306/18709
16307/18709
16308/18709
16309/18709
16310/18709
16311/18709
16312/18709
16313/18709
16314/18709
16315/18709
16316/18709
16317/18709
16318/18709
16319/18709
16320/18709
16321/18709
16322/18709
16323/18709
16324/18709
16325/18709
16326/18709
16327/18709
16328/18709
16329/18709
16330/18709
16331/18709
16332/18709
16333/18709
16334/18709
16335/18709
16336/18709
16337/18709
16338/18709
16339/18709
16340/18709
16341/18709
16342/18709
16343/18709
16344/18709
16345/18709
16346/18709
16347/18709
16348/18709
16349/18709
16350/18709
16351/18709
16352/18709
16353/18709
16354/18709
16355/18709
1635

16985/18709
16986/18709
16987/18709
16988/18709
16989/18709
16990/18709
16991/18709
16992/18709
16993/18709
16994/18709
16995/18709
16996/18709
16997/18709
16998/18709
16999/18709
17000/18709
17001/18709
17002/18709
17003/18709
17004/18709
17005/18709
17006/18709
17007/18709
17008/18709
17009/18709
17010/18709
17011/18709
17012/18709
17013/18709
17014/18709
17015/18709
17016/18709
17017/18709
17018/18709
17019/18709
17020/18709
17021/18709
17022/18709
17023/18709
17024/18709
17025/18709
17026/18709
17027/18709
17028/18709
17029/18709
17030/18709
17031/18709
17032/18709
17033/18709
17034/18709
17035/18709
17036/18709
17037/18709
17038/18709
17039/18709
17040/18709
17041/18709
17042/18709
17043/18709
17044/18709
17045/18709
17046/18709
17047/18709
17048/18709
17049/18709
17050/18709
17051/18709
17052/18709
17053/18709
17054/18709
17055/18709
17056/18709
17057/18709
17058/18709
17059/18709
17060/18709
17061/18709
17062/18709
17063/18709
17064/18709
17065/18709
17066/18709
17067/18709
1706

17692/18709
17693/18709
17694/18709
17695/18709
17696/18709
17697/18709
17698/18709
17699/18709
17700/18709
17701/18709
17702/18709
17703/18709
17704/18709
17705/18709
17706/18709
17707/18709
17708/18709
17709/18709
17710/18709
17711/18709
17712/18709
17713/18709
17714/18709
17715/18709
17716/18709
17717/18709
17718/18709
17719/18709
17720/18709
17721/18709
17722/18709
17723/18709
17724/18709
17725/18709
17726/18709
17727/18709
17728/18709
17729/18709
17730/18709
17731/18709
17732/18709
17733/18709
17734/18709
17735/18709
17736/18709
17737/18709
17738/18709
17739/18709
17740/18709
17741/18709
17742/18709
17743/18709
17744/18709
17745/18709
17746/18709
17747/18709
17748/18709
17749/18709
17750/18709
17751/18709
17752/18709
17753/18709
17754/18709
17755/18709
17756/18709
17757/18709
17758/18709
17759/18709
17760/18709
17761/18709
17762/18709
17763/18709
17764/18709
17765/18709
17766/18709
17767/18709
17768/18709
17769/18709
17770/18709
17771/18709
17772/18709
17773/18709
17774/18709
1777

18390/18709
18391/18709
18392/18709
18393/18709
18394/18709
18395/18709
18396/18709
18397/18709
18398/18709
18399/18709
18400/18709
18401/18709
18402/18709
18403/18709
18404/18709
18405/18709
18406/18709
18407/18709
18408/18709
18409/18709
18410/18709
18411/18709
18412/18709
18413/18709
18414/18709
18415/18709
18416/18709
18417/18709
18418/18709
18419/18709
18420/18709
18421/18709
18422/18709
18423/18709
18424/18709
18425/18709
18426/18709
18427/18709
18428/18709
18429/18709
18430/18709
18431/18709
18432/18709
18433/18709
18434/18709
18435/18709
18436/18709
18437/18709
18438/18709
18439/18709
18440/18709
18441/18709
18442/18709
18443/18709
18444/18709
18445/18709
18446/18709
18447/18709
18448/18709
18449/18709
18450/18709
18451/18709
18452/18709
18453/18709
18454/18709
18455/18709
18456/18709
18457/18709
18458/18709
18459/18709
18460/18709
18461/18709
18462/18709
18463/18709
18464/18709
18465/18709
18466/18709
18467/18709
18468/18709
18469/18709
18470/18709
18471/18709
18472/18709
1847